# The Value of Cultural Similarity for Predicting Migration


## Carolina Coimbra Vieira, Sophie Lohmann, Emilio Zagheni

In [1]:
shhh <- suppressPackageStartupMessages # It's a library, so shhh!

shhh(library(tidyr))
shhh(library(dplyr))
shhh(library(readr))
shhh(library(lsa))

Warning message:
“replacing previous import ‘lifecycle::last_warnings’ by ‘rlang::last_warnings’ when loading ‘pillar’”Warning message:
“replacing previous import ‘lifecycle::last_warnings’ by ‘rlang::last_warnings’ when loading ‘tibble’”

## Population 2019
### Datasource: [UN](https://population.un.org/wpp/Download/Standard/Population/)

In [2]:
pop_2019 <- read_csv("data/subset-pop-un2019.csv", col_types = cols())

head(pop_2019)

country,population
Argentina,44780675
Australia,25203200
Brazil,211049519
Chile,18952035
France,65129731
Great Britain,67530161


In [3]:
countries <- c(pop_2019$country)

countries

[1] "Argentina"     "Australia"     "Brazil"        "Chile"        
 [5] "France"        "Great Britain" "Indonesia"     "Japan"        
 [9] "Malaysia"      "Mexico"        "Russia"        "Singapore"    
[13] "South Korea"   "Spain"         "Turkey"        "United States"

## Geographic info: area  
### Datasource: [CEPII](http://www.cepii.fr/CEPII/en/bdd_modele/presentation.asp?id=6)

<b>Country-level variables</b>   
    *`iso2, iso3, cnum`*: ISO codes in two and three characters, and in three numbers respectively.  
    *`country, pays`*: Name of country in English and French respectively.   
    **`area`: Country’s area in $km^2$.**   
    *`dis_int`*: Internal distance of country i, $d_{ii} = .67 \sqrt{area/\pi}$ (an often used measure of average distance between producers and consumers in a country, see Head and Mayer, 2002 for more on this topic).  
    *`landlocked`*: Dummy variable set equal to 1 for landlocked countries.    
    *`continent`*: Continent to which the country is belonging.   
    *`langoff_i`*: Official or national languages and languages spoken by at least 20% of the population of the country (and spoken in another country of the world) following the same logic than the "open-circuit languages" in Mélitz (2002).  
    *`lang20_i`*: Languages (mother tongue, lingua francas or second languages) spoken by at least 20% of the population of the country.  
    *`lang9_i`*: Languages (mother tongue, lingua francas or second languages) spoken by between 9% and 20% of the population of the country.  
    *`colonizeri`*: Colonizers of the country for a relatively long period of time and with a substantial participation in the governance of the colonized country.  
    *`short_colonizeri`*: Colonizers of the country for a relatively short period of time or with only low involvement in the governance of the colonized country.
    
<b>Cities variables used in the computation of distances</b>  
    *`city_en, city_fr`*: Names of capitals or main cities of the country in English and French.   
    *`lat, lon`*: Latitude and longitude of the city.  
    *`cap`*: Variable equals to 1 if the city is the capital of the country, to 0 if the city is the most populated city (maincity equals to 1) but not the capital, and to 2 in the cases of two capitals, if the city is the most populated but the “second” capital or the previous capital.  
    *`maincity`*: Variable coded as 1 when the city is the most populated of the country and as 2 otherwise.  
    *`citynum`*: Number of cities for each country used to calculate our weighted distances described in the next section.  

In [4]:
geo <- read.csv("data/cepii_geo.csv")

head(geo)

iso2,iso3,cnum,country,pays,area,dis_int,landlocked,continent,city_en,...,lang9_2,lang9_3,lang9_4,colonizer1,colonizer2,colonizer3,colonizer4,short_colonizer1,short_colonizer2,short_colonizer3
AW,ABW,533,Aruba,Aruba,193,"5,225315",0,America,Oranjestad,...,.,.,.,NLD,.,.,.,.,.,.
AF,AFG,4,Afghanistan,Afghanistan,652225,"303,7614",1,Asia,Kabul,...,Uzbek,.,.,.,.,.,.,GBR,.,.
AO,AGO,24,Angola,Angola,1246700,"419,9666",0,Africa,Luanda,...,.,.,.,PRT,.,.,.,.,.,.
AI,AIA,660,Anguilla,Anguilla,102,"3,79869",0,America,The Valley,...,.,.,.,GBR,.,.,.,.,.,.
AL,ALB,8,Albania,Albanie,28748,"63,77311",0,Europe,Tirana,...,.,.,.,TUR,.,.,.,.,.,.
AD,AND,20,Andorra,Andorre,453,"8,005398",0,Europe,Andorra la Vella,...,.,.,.,.,.,.,.,.,.,.


In [5]:
colnames(geo)

[1] "iso2"             "iso3"             "cnum"             "country"         
 [5] "pays"             "area"             "dis_int"          "landlocked"      
 [9] "continent"        "city_en"          "city_fr"          "lat"             
[13] "lon"              "cap"              "maincity"         "citynum"         
[17] "langoff_1"        "langoff_2"        "langoff_3"        "lang20_1"        
[21] "lang20_2"         "lang20_3"         "lang20_4"         "lang9_1"         
[25] "lang9_2"          "lang9_3"          "lang9_4"          "colonizer1"      
[29] "colonizer2"       "colonizer3"       "colonizer4"       "short_colonizer1"
[33] "short_colonizer2" "short_colonizer3"

In [6]:
area <- geo %>% filter (country %in% countries) %>%
                select(iso2, iso3, country, area) %>% 
                unique()

head(area)

,iso2,iso3,country,area
1,AR,ARG,Argentina,2766889
2,AU,AUS,Australia,7686848
4,BR,BRA,Brazil,8511920
6,CL,CHL,Chile,756945
7,ES,ESP,Spain,505954
8,FR,FRA,France,547026


#### Population + Area

In [7]:
area_pop <- merge(area, pop_2019)

area_pop

country,iso2,iso3,area,population
Argentina,AR,ARG,2766889,44780675
Australia,AU,AUS,7686848,25203200
Brazil,BR,BRA,8511920,211049519
Chile,CL,CHL,756945,18952035
France,FR,FRA,547026,65129731
Great Britain,GB,GBR,244110,67530161
Indonesia,ID,IDN,1933658,270625567
Japan,JP,JPN,377801,126860299
Malaysia,MY,MYS,329758,31949789
Mexico,MX,MEX,1967210,127575529


In [8]:
isos3 <- area$iso3

isos3

[1] ARG AUS BRA CHL ESP FRA GBR IDN JPN KOR MEX MYS RUS SGP TUR USA
225 Levels: ABW AFG AGO AIA ALB AND ANT ARE ARG ARM ATF ATG AUS AUT AZE ... ZWE

## Geographic distance
### Datasource: [CEPII](http://www.cepii.fr/CEPII/en/bdd_modele/presentation.asp?id=6)  

<b>Variables</b>  
    *`iso_o, iso_d`*: ISO codes in three characters, refering to the country of origin and destination, respectively.  
    *`dist`*: Geodesic distances from lat/long of most populous cities (simple distance).   
    *`distcap`*: geodesic distance between capital cities (simple distance).    
    *`distw`*: population weighted distance, theta = 1 (weighted distance).   
    *`distwces`*: population weighted distance, theta = -1, which corresponds to the usual coefficient estimated from gravity models of bilateral trade flows (weighted distance).  
    *`contig`*: dummy variable indicating whether the two countries are contiguous (share a land border).  
    *`comlang_off, comlang_ethno`*: dummy variable indicating whether the two countries share a common language. There are two common languages dummies, the first one based on the fact that two countries share a common official language, and the other one set to one if a language is spoken by at least 9% of the population in both countries.     
    *`colony`*: dummy variable indicating whether the two countries have ever had a colonial link.  
    *`comcol`*: dummy variable indicating whether the two countries have had a common colonizer after 1945.  
    *`curcol`*: dummy variable indicating whether the two countries are currently in a colonial relationship.  
    *`col45`*: dummy variable indicating whether the two countries have had a colonial relationship -a-f-t-e-r- 1945 (share common colonizer pre 1945).  
    *`smctry`*: dummy variable indicating whether the two countries were/are the same country.  

**[TO DO]: check Maciej distance**   
<b>Maciej</b>  
    *`dist_pop_weighted`*: population-weighted average distance between biggest cities  
    *`dist_biggest_cities`*: average distance between biggest cities  
^ most similar to distwces  
    *`dist_unweighted`*: average distance between (?) (not population weighted)  

In [9]:
dists <- read.csv("data/cepii_dist.csv")

head(dists)

iso_o,iso_d,contig,comlang_off,comlang_ethno,colony,comcol,curcol,col45,smctry,dist,distcap,distw,distwces
ABW,ABW,0,0,0,0,0,0,0,0,"5,225315","5,225315","25,09354","23,04723"
ABW,AFG,0,0,0,0,0,0,0,0,"13257,81","13257,81","13168,22","13166,37"
ABW,AGO,0,0,0,0,0,0,0,0,"9516,913","9516,913","9587,316","9584,193"
ABW,AIA,0,0,1,0,0,0,0,0,"983,2682","983,2682","976,8974","976,8916"
ABW,ALB,0,0,0,0,0,0,0,0,"9091,742","9091,742","9091,576","9091,466"
ABW,AND,0,1,0,0,0,0,0,0,"7572,788","7572,788","7570,084","7570,083"


In [10]:
dists <- dists %>% filter (iso_o %in% isos3 & iso_d %in% isos3)

head(dists)

iso_o,iso_d,contig,comlang_off,comlang_ethno,colony,comcol,curcol,col45,smctry,dist,distcap,distw,distwces
ARG,ARG,0,0,0,0,0,0,0,0,"625,6475","625,6475","533,9082","96,14934"
ARG,AUS,0,0,0,0,0,0,0,0,"11801,36","11733,88","12044,57","12018,27"
ARG,BRA,1,0,0,0,0,0,0,0,"1691,067","2353,257","2391,846","2089,281"
ARG,CHL,1,1,1,0,0,0,0,0,"1128,317","1128,317","1156,726","941,6383"
ARG,ESP,0,1,1,1,0,0,0,0,"10065,85","10065,85","10079,66","10055,9"
ARG,FRA,0,0,0,0,0,0,0,0,"11072,25","11072,25","10932,34","10921,86"


**New variable:**  
    *`shared_hist`*: dummy variable indicating whether the two countries have: *`colony, comcol, curcol, col45, smctry`*

In [11]:
dists <- dists %>% mutate(shared_hist = ifelse(colony + comcol + curcol + col45 + smctry, 1, 0), .keep = "unused")

head(dists)

iso_o,iso_d,contig,comlang_off,comlang_ethno,dist,distcap,distw,distwces,shared_hist
ARG,ARG,0,0,0,"625,6475","625,6475","533,9082","96,14934",0
ARG,AUS,0,0,0,"11801,36","11733,88","12044,57","12018,27",0
ARG,BRA,1,0,0,"1691,067","2353,257","2391,846","2089,281",0
ARG,CHL,1,1,1,"1128,317","1128,317","1156,726","941,6383",0
ARG,ESP,0,1,1,"10065,85","10065,85","10079,66","10055,9",1
ARG,FRA,0,0,0,"11072,25","11072,25","10932,34","10921,86",0


#### Population + Area + Geographic distance  + Shared history

In [12]:
area_pop_gd_d <- area_pop %>% select(!iso2) %>%
                               merge(dists, by.x=c("iso3"), by.y=c("iso_d")) %>%
                               rename(iso_d = iso3, destination = country, area_d = area, pop_d = population) 

area_pop_gd_o <- area_pop %>% select(!iso2) %>% 
                               merge(dists, by.x=c("iso3"), by.y=c("iso_o")) %>%
                               rename(iso_o = iso3, origin = country, area_o = area, pop_o = population) 

area_pop_gdist <- merge(area_pop_gd_d, area_pop_gd_o)

head(area_pop_gdist)

iso_d,iso_o,contig,comlang_off,comlang_ethno,dist,distcap,distw,distwces,shared_hist,destination,area_d,pop_d,origin,area_o,pop_o
ARG,ARG,0,0,0,"625,6475","625,6475","533,9082","96,14934",0,Argentina,2766889,44780675,Argentina,2766889,44780675
ARG,AUS,0,0,0,"11801,36","11733,88","12044,57","12018,27",0,Argentina,2766889,44780675,Australia,7686848,25203200
ARG,BRA,1,0,0,"1691,067","2353,257","2391,846","2089,281",0,Argentina,2766889,44780675,Brazil,8511920,211049519
ARG,CHL,1,1,1,"1128,317","1128,317","1156,726","941,6383",0,Argentina,2766889,44780675,Chile,756945,18952035
ARG,ESP,0,1,1,"10065,85","10065,85","10079,66","10055,9",1,Argentina,2766889,44780675,Spain,505954,46736782
ARG,FRA,0,0,0,"11072,25","11072,25","10932,34","10921,86",0,Argentina,2766889,44780675,France,547026,65129731


## Language  
### Datasource: [CEPII](http://www.cepii.fr/CEPII/en/bdd_modele/presentation.asp?id=19)   

<b>Variables [Sarah's notes]</b>  
    *`col`*: common official language (0 or 1); 19 languages considered.    
    *`csl`*: p(two random people understand a common language) >= *`cnl`*.  
    *`cnl`*: p(two random people share a native language).  
    *`lp`*: lexical closeness of native langauges; set to 0 when *`cnl`* is 1 or 0 also set to 0 if there is no dominant native language (e.g. India).  
    *`lp1`*: tree based. 4 possibilities, 2 languages belonging to:  
        0: separate family trees  
        0.25: different branches of same tree (English and French),  
        0.50: the same branch (English and German),  
        0.75: the same sub-branch (German and Dutch)  
    *`lp2`*: lexical similarity of 200 words, continuous scale 0-100 normalized *`lp1`*, *`lp2`* so coefficients are comparable to each other and *`col, prox1, prox2`* are unadjusted versions of *`lp1`* and *`lp2`*?  
  

In [13]:
langs <- read.csv("data/cepii_language.csv")

head(langs)

X,iso_o,country_o,iso_d,country_d,col,csl,cnl,prox1,lp1,prox2,lp2,cl,cle
1,AFG,Afghanistan,ALB,Albania,0,0,0,0.25,NA,0.07342058,NA,NA,0.08662836
2,AFG,Afghanistan,DZA,Algeria,0,0,0,0.00,0.000000,0.08527792,0.7640970,0.09992715,0.10061873
3,AFG,Afghanistan,AND,Andorra,0,0,0,0.25,1.945866,0.12025990,1.0775385,0.14091842,0.14189370
4,AFG,Afghanistan,AGO,Angola,0,0,0,0.25,NA,0.11890529,NA,NA,0.14029542
5,AFG,Afghanistan,AIA,Anguilla,0,0,0,0.25,NA,0.09291404,NA,NA,0.10962854
6,AFG,Afghanistan,ATG,Antigua and Barbuda,0,0,0,0.25,1.945866,0.09291404,0.8325174,0.10887503,0.10962854


In [14]:
langs <- langs %>% filter (iso_o %in% isos3 & iso_d %in% isos3)

head(langs)

X,iso_o,country_o,iso_d,country_d,col,csl,cnl,prox1,lp1,prox2,lp2,cl,cle
1368,ARG,Argentina,AUS,Australia,0,0.02058416,0.00080000,0.25,1.945866,0.13052547,1.1695188,0.15362507,0.15468276
1383,ARG,Argentina,BRA,Brazil,0,0.06052872,0.00080000,0.75,5.837599,0.42439765,3.8026376,0.49770382,0.50114280
1396,ARG,Argentina,CHL,Chile,1,0.98010004,0.85439998,0.00,0.000000,0.00000000,0.0000000,0.87344128,0.87256539
1422,ARG,Argentina,FRA,France,0,0.13392781,0.01039232,0.50,3.891733,0.25450581,2.2803929,0.30551848,0.30756098
1441,ARG,Argentina,IDN,Indonesia,0,0.00000000,0.00000000,0.00,0.000000,0.07436915,0.6663537,0.08714446,0.08774757
1448,ARG,Argentina,JPN,Japan,0,0.00000000,0.00000000,0.00,0.000000,0.06896222,0.6179070,0.08080870,0.08136797


#### Population + Area + Geographic distance + Shared history +  Language

In [15]:
area_pop_gdist_lang <- langs %>% select(!c(X, country_o, country_d)) %>%
                                merge(area_pop_gdist)

head(area_pop_gdist_lang)

iso_o,iso_d,col,csl,cnl,prox1,lp1,prox2,lp2,cl,...,distcap,distw,distwces,shared_hist,destination,area_d,pop_d,origin,area_o,pop_o
ARG,AUS,0,0.02058416,0.00080000,0.25,1.945866,0.1305255,1.169519,0.1536251,...,"11733,88","12044,57","12018,27",0,Australia,7686848,25203200,Argentina,2766889,44780675
ARG,BRA,0,0.06052872,0.00080000,0.75,5.837599,0.4243976,3.802638,0.4977038,...,"2353,257","2391,846","2089,281",0,Brazil,8511920,211049519,Argentina,2766889,44780675
ARG,CHL,1,0.98010004,0.85439998,0.00,0.000000,0.0000000,0.000000,0.8734413,...,"1128,317","1156,726","941,6383",0,Chile,756945,18952035,Argentina,2766889,44780675
ARG,ESP,1,0.98013985,0.85445821,0.00,0.000000,0.0000000,0.000000,0.8734919,...,"10065,85","10079,66","10055,9",1,Spain,505954,46736782,Argentina,2766889,44780675
ARG,FRA,0,0.13392781,0.01039232,0.50,3.891733,0.2545058,2.280393,0.3055185,...,"11072,25","10932,34","10921,86",0,France,547026,65129731,Argentina,2766889,44780675
ARG,GBR,0,0.08325152,0.00000000,0.25,1.945866,0.1305255,1.169519,0.1529474,...,"11147,48","11136,96","11128,22",0,Great Britain,244110,67530161,Argentina,2766889,44780675


## GDP (constant 2010) 2019  
### Datasource: [World Bank](https://databank.worldbank.org/source/world-development-indicators)

*`GDP`*: GDP (constant 2010) in 2019 in US$

GDP at purchaser's prices is the sum of gross value added by all resident producers in the economy plus any product taxes and minus any subsidies not included in the value of the products. It is calculated without making deductions for depreciation of fabricated assets or for depletion and degradation of natural resources. Data are in constant 2010 U.S. dollars. Dollar figures for GDP are converted from domestic currencies using 2010 official exchange rates. For a few countries where the official exchange rate does not reflect the rate effectively applied to actual foreign exchange transactions, an alternative conversion factor is used.

In [16]:
gdp <- read_csv("data/subset-GDP_constant2010-wb2019.csv", col_types = cols())

head(gdp)

iso_country,GDP
AUS,1.450499e+12
ARG,4.378134e+11
BRA,2.364446e+12
CHL,2.850370e+11
FRA,2.971919e+12
IDN,1.204457e+12


In [17]:
area_pop_gdist_lang_gdp <- area_pop_gdist_lang %>% 
                               merge(gdp, by.x=c("iso_o"), by.y=c("iso_country")) %>% 
                               rename(GDP_o = GDP) %>%
                               merge(gdp, by.x=c("iso_d"), by.y=c("iso_country")) %>%
                               rename(GDP_d = GDP)
                               
head(area_pop_gdist_lang_gdp)

iso_d,iso_o,col,csl,cnl,prox1,lp1,prox2,lp2,cl,...,distwces,shared_hist,destination,area_d,pop_d,origin,area_o,pop_o,GDP_o,GDP_d
ARG,CHL,1,0.98010004,0.8544,0.00,0.000000,0.0000000,0.000000,0.8734413,...,"941,6383",0,Argentina,2766889,44780675,Chile,756945,18952035,2.850370e+11,437813398163
ARG,GBR,0,0.08325152,0.0000,0.25,1.945866,0.1305255,1.169519,0.1529474,...,"11128,22",0,Argentina,2766889,44780675,Great Britain,244110,67530161,2.913557e+12,437813398163
ARG,AUS,0,0.02058416,0.0008,0.25,1.945866,0.1305255,1.169519,0.1536251,...,"12018,27",0,Argentina,2766889,44780675,Australia,7686848,25203200,1.450499e+12,437813398163
ARG,KOR,0,0.00000000,0.0000,0.00,0.000000,0.0000000,0.000000,0.0000000,...,"19137,75",0,Argentina,2766889,44780675,South Korea,99484,51225321,1.482760e+12,437813398163
ARG,BRA,0,0.06052872,0.0008,0.75,5.837599,0.4243976,3.802638,0.4977038,...,"2089,281",0,Argentina,2766889,44780675,Brazil,8511920,211049519,2.364446e+12,437813398163
ARG,MEX,1,0.98010004,0.8832,0.00,0.000000,0.0000000,0.000000,0.8984749,...,"7460,572",0,Argentina,2766889,44780675,Mexico,1967210,127575529,1.309882e+12,437813398163


## Migration (stock) 2019  
### Datasource: [UN](https://www.un.org/en/development/desa/population/migration/data/estimates2/estimates19.asp)

The dataset presents estimates of international migrant by age, sex and origin. Estimates are presented for 1990, 1995, 2000, 2005, 2010, 2015 and 2019 and are available for all countries and areas of the world. The estimates are based on official statistics on the foreign-born or the foreign population.   
International migrant stock - Total international migrant stock   

In [18]:
migration_stock_wide <- read.csv("data/subset-stocks-migration-un2019.csv")
migration_stock_wide[is.na(migration_stock_wide)] <- 0

head(migration_stock_wide)

destination,Total,Total16,Argentina,Australia,Brazil,Chile,France,Great.Britain,Indonesia,Japan,Malaysia,Mexico,Russia,Singapore,South.Korea,Spain,Turkey,United.States
Argentina,2212879,397927,0,279,49647,216855,8303,959,21,4531,2,1475,1917,2,8296,99769,646,5225
Australia,7549270,2109825,17583,0,42552,33707,40733,1262204,87274,53175,174136,6760,28735,64739,121204,19594,42880,114549
Brazil,807006,229897,39078,791,0,20747,12851,6650,978,65955,163,3601,1865,154,11531,41321,486,23726
Chile,939992,177893,72813,3952,20370,0,12562,6668,343,3231,100,6887,2344,237,4247,22842,1019,20278
France,8334875,1110686,14253,9192,63208,15857,0,176672,5446,23536,2793,13851,70463,2512,22794,303245,327508,59356
Great Britain,9552110,1073094,12239,145692,47487,8668,170142,0,11543,52930,84638,10457,38917,58432,13171,98372,104491,215915


In [19]:
migration_stock <- migration_stock_wide %>% 
                pivot_longer(!c(destination, Total, Total16), names_to = "origin", values_to = "m_stock") %>%
                select(destination, Total, origin, m_stock) %>%
                rename(total_imm_in_d = Total)
migration_stock$origin <- chartr(".", " ", migration_stock$origin)

head(migration_stock)

destination,total_imm_in_d,origin,m_stock
Argentina,2212879,Argentina,0
Argentina,2212879,Australia,279
Argentina,2212879,Brazil,49647
Argentina,2212879,Chile,216855
Argentina,2212879,France,8303
Argentina,2212879,Great Britain,959


In [20]:
area_pop_mig_d <- area_pop %>% select(country, iso3) %>% 
                               merge(migration_stock, by.x=c("country"), by.y=c("destination")) %>%
                               rename(iso_d = iso3, destination = country) 

area_pop_mig_o <- area_pop %>% select(country, iso3) %>% 
                               merge(migration_stock, by.x=c("country"), by.y=c("origin")) %>%
                               rename(iso_o = iso3, origin = country) 

migration_stock <- merge(area_pop_mig_d, area_pop_mig_o)

head(migration_stock)

destination,total_imm_in_d,origin,m_stock,iso_d,iso_o
Argentina,2212879,Argentina,0,ARG,ARG
Argentina,2212879,Australia,279,ARG,AUS
Argentina,2212879,Brazil,49647,ARG,BRA
Argentina,2212879,Chile,216855,ARG,CHL
Argentina,2212879,France,8303,ARG,FRA
Argentina,2212879,Great Britain,959,ARG,GBR


In [21]:
official_dataset <- merge(area_pop_gdist_lang_gdp, migration_stock) %>% select(!total_imm_in_d)

head(official_dataset)

iso_d,iso_o,destination,origin,col,csl,cnl,prox1,lp1,prox2,...,distw,distwces,shared_hist,area_d,pop_d,area_o,pop_o,GDP_o,GDP_d,m_stock
ARG,AUS,Argentina,Australia,0,0.02058416,0.00080000,0.25,1.945866,0.1305255,...,"12044,57","12018,27",0,2766889,44780675,7686848,25203200,1.450499e+12,437813398163,279
ARG,BRA,Argentina,Brazil,0,0.06052872,0.00080000,0.75,5.837599,0.4243976,...,"2391,846","2089,281",0,2766889,44780675,8511920,211049519,2.364446e+12,437813398163,49647
ARG,CHL,Argentina,Chile,1,0.98010004,0.85439998,0.00,0.000000,0.0000000,...,"1156,726","941,6383",0,2766889,44780675,756945,18952035,2.850370e+11,437813398163,216855
ARG,ESP,Argentina,Spain,1,0.98013985,0.85445821,0.00,0.000000,0.0000000,...,"10079,66","10055,9",1,2766889,44780675,505954,46736782,1.572013e+12,437813398163,99769
ARG,FRA,Argentina,France,0,0.13392781,0.01039232,0.50,3.891733,0.2545058,...,"10932,34","10921,86",0,2766889,44780675,547026,65129731,2.971919e+12,437813398163,8303
ARG,GBR,Argentina,Great Britain,0,0.08325152,0.00000000,0.25,1.945866,0.1305255,...,"11136,96","11128,22",0,2766889,44780675,244110,67530161,2.913557e+12,437813398163,959


In [22]:
dim(official_dataset)

[1] 240  28

## Migration (flow) 2015-2020  
### Datasource: [Abel, Guy; E. Cohen, Joel (2019)](https://figshare.com/collections/Bilateral_international_migration_flow_estimates_for_200_countries/4470464)

This collection contains bilateral international migration flow estimates for 200 countries for five-year periods between 1990 and 2020. 

In [23]:
migration_flow <- read.csv("data/abel_cohen_migration_flow.csv")
migration_flow[is.na(migration_flow)] <- 0

head(migration_flow)

year0,orig,dest,sd_drop_neg,sd_rev_neg,mig_rate,da_min_open,da_min_closed,da_pb_closed
1990,BDI,BDI,0,0,0,0,0,0
1990,COM,BDI,0,0,0,0,0,0
1990,DJI,BDI,0,0,0,0,0,0
1990,ERI,BDI,0,0,0,0,0,90
1990,ETH,BDI,0,0,0,0,0,2
1990,KEN,BDI,30,30,69,45,29,87


In [24]:
migration_flow <- migration_flow %>% 
                filter(year0 == 2015) %>%
                select(!year0) %>%
                rename(iso_d = dest, iso_o = orig, 
                       m_flow_sd_drop_neg = sd_drop_neg, 
                       m_flow_sd_rev_neg = sd_rev_neg, 
                       m_flow_mig_rate = mig_rate, 
                       m_flow_da_min_open = da_min_open, 
                       m_flow_da_min_closed = da_min_closed, 
                       m_flow_da_pb_closed = da_pb_closed)

head(migration_flow)

iso_o,iso_d,m_flow_sd_drop_neg,m_flow_sd_rev_neg,m_flow_mig_rate,m_flow_da_min_open,m_flow_da_min_closed,m_flow_da_pb_closed
BDI,BDI,0,0,0,0,0,0
COM,BDI,0,0,0,0,0,0
DJI,BDI,0,0,0,0,0,0
ERI,BDI,0,131,0,43,60,183
ETH,BDI,0,14,0,9,3,24
KEN,BDI,194,194,211,1168,447,1554


In [25]:
official_dataset <- merge(official_dataset, migration_flow)

head(official_dataset)

iso_d,iso_o,destination,origin,col,csl,cnl,prox1,lp1,prox2,...,pop_o,GDP_o,GDP_d,m_stock,m_flow_sd_drop_neg,m_flow_sd_rev_neg,m_flow_mig_rate,m_flow_da_min_open,m_flow_da_min_closed,m_flow_da_pb_closed
ARG,AUS,Argentina,Australia,0,0.02058416,0.00080000,0.25,1.945866,0.1305255,...,25203200,1.450499e+12,437813398163,279,1,1,56,16,105,2224
ARG,BRA,Argentina,Brazil,0,0.06052872,0.00080000,0.75,5.837599,0.4243976,...,211049519,2.364446e+12,437813398163,49647,475,475,9997,2332,0,10767
ARG,CHL,Argentina,Chile,1,0.98010004,0.85439998,0.00,0.000000,0.0000000,...,18952035,2.850370e+11,437813398163,216855,2079,2079,43666,13875,53604,84213
ARG,ESP,Argentina,Spain,1,0.98013985,0.85445821,0.00,0.000000,0.0000000,...,46736782,1.572013e+12,437813398163,99769,956,956,20090,6434,5516,49532
ARG,FRA,Argentina,France,0,0.13392781,0.01039232,0.50,3.891733,0.2545058,...,65129731,2.971919e+12,437813398163,8303,79,79,1672,467,156,3048
ARG,GBR,Argentina,Great Britain,0,0.08325152,0.00000000,0.25,1.945866,0.1305255,...,67530161,2.913557e+12,437813398163,959,8,8,193,45,126,1559


In [26]:
dim(official_dataset)

[1] 240  34

## WVS
### Datasource: [WVS (2020)](https://www.worldvaluessurvey.org/wvs.jsp)



In [27]:
wvs <- read.csv("data/subset-wvs-wave7-2020.csv")

wvs_cosine <- as.data.frame(cosine(t(as.matrix(wvs[,2:3]))))
colnames(wvs_cosine) <- wvs$country
wvs_cosine["origin"] <- wvs$country


wvs_cosine_longer <- pivot_longer(wvs_cosine, -c("origin"), values_to = "CS_wvs", names_to = "destination")
head(wvs_cosine_longer)

origin,destination,CS_wvs
Argentina,Argentina,1.0000000
Argentina,Australia,0.2579213
Argentina,Brazil,0.7688173
Argentina,Chile,-0.9899189
Argentina,France,0.1697698
Argentina,Great Britain,0.3229716


## Foursquare
### Datasource: [You Are What You Eat (and Drink): Identifying Cultural Boundaries by Analyzing Food and Drink Habits in Foursquare. Thiago H. Silva, Pedro O. S. Vaz de Melo, Jussara M. Almeida, Mirco Musolesi, Antonio A. F. Loureiro (2014)](https://www.aaai.org/ocs/index.php/ICWSM/ICWSM14/paper/viewPaper/8113)



In [28]:
foursquare <- read.csv("data/foursquare.csv")

foursquare_cosine <- as.data.frame(cosine(t(as.matrix(foursquare[,2:3]))))
colnames(foursquare_cosine) <- foursquare$country
foursquare_cosine["origin"] <- foursquare$country


foursquare_cosine_longer <- pivot_longer(foursquare_cosine, -c("origin"), values_to = "CS_foursquare", names_to = "destination")
head(foursquare_cosine_longer)

origin,destination,CS_foursquare
Argentina,Argentina,1.0000000
Argentina,Australia,0.1005334
Argentina,Brazil,0.8402622
Argentina,Chile,0.9548769
Argentina,France,0.8741573
Argentina,Great Britain,0.6263188


In [29]:
official_dataset <- merge(official_dataset, wvs_cosine_longer)
official_dataset <- merge(official_dataset, foursquare_cosine_longer)

head(official_dataset)

destination,origin,iso_d,iso_o,col,csl,cnl,prox1,lp1,prox2,...,GDP_d,m_stock,m_flow_sd_drop_neg,m_flow_sd_rev_neg,m_flow_mig_rate,m_flow_da_min_open,m_flow_da_min_closed,m_flow_da_pb_closed,CS_wvs,CS_foursquare
Argentina,Australia,ARG,AUS,0,0.02058416,0.00080000,0.25,1.945866,0.13052547,...,437813398163,279,1,1,56,16,105,2224,0.2579213,0.1005334
Argentina,Brazil,ARG,BRA,0,0.06052872,0.00080000,0.75,5.837599,0.42439765,...,437813398163,49647,475,475,9997,2332,0,10767,0.7688173,0.8402622
Argentina,Chile,ARG,CHL,1,0.98010004,0.85439998,0.00,0.000000,0.00000000,...,437813398163,216855,2079,2079,43666,13875,53604,84213,-0.9899189,0.9548769
Argentina,France,ARG,FRA,0,0.13392781,0.01039232,0.50,3.891733,0.25450581,...,437813398163,8303,79,79,1672,467,156,3048,0.1697698,0.8741573
Argentina,Great Britain,ARG,GBR,0,0.08325152,0.00000000,0.25,1.945866,0.13052547,...,437813398163,959,8,8,193,45,126,1559,0.3229716,0.6263188
Argentina,Indonesia,ARG,IDN,0,0.00000000,0.00000000,0.00,0.000000,0.07436915,...,437813398163,21,0,0,4,1,4,7,0.4656905,-0.8793284


In [30]:
dim(official_dataset)

[1] 240  36

In [31]:
write_csv(official_dataset, "data/subset-official-dataset-05-22.csv")

## Cultural Similarity 

#### symmetric:   

each country is represented by a vector corresponding to the top (almost) <b>400 dishes</b> (top 50 dishes in all the countries)

![CD](figs/heatmap-similarity-symmetric-tops-topK-cosine-reds.png)

In [32]:
cd_sym <- read_csv("data/FB-food-cultural_similarity-symmetric-top-163.csv", col_types = cols())
colnames(cd_sym)[1] <- "interests_from"

Warning message:
“Missing column names filled in: 'X1' [1]”

In [33]:
cd_sym

interests_from,Argentina,Australia,Brazil,Chile,France,Great Britain,Indonesia,Japan,Malaysia,Mexico,Russia,Singapore,South Korea,Spain,Turkey,United States
Argentina,1.000000000,-0.1626831,0.19485541,0.41201274,-0.003257801,-0.17993609,-0.13704314,-0.13214747,-0.26977055,0.29775594,-0.04100249,-0.2566822,-0.16094402,0.20017383,-0.06210403,-0.15866855
Australia,-0.162683113,1.0000000,-0.23317915,-0.29450338,-0.287144698,0.42500589,-0.51754757,-0.54966290,0.34170015,-0.13035142,-0.61086630,0.3371980,-0.32945662,-0.34235562,-0.42960443,0.54298889
Brazil,0.194855413,-0.2331792,1.00000000,0.17574451,0.060125106,-0.10884516,0.01269456,0.12001132,-0.32225425,0.02640051,0.13359658,-0.3050608,-0.05114448,0.12168753,0.03903992,-0.21211853
Chile,0.412012736,-0.2945034,0.17574451,1.00000000,0.031023883,-0.21033239,-0.04680964,0.04865095,-0.36937776,0.33912740,0.10013607,-0.3215099,-0.12849570,0.25108751,0.03050597,-0.20420369
France,-0.003257801,-0.2871447,0.06012511,0.03102388,1.000000000,-0.09859411,0.13017437,0.23963749,-0.37828174,-0.08554674,0.33588439,-0.3263162,-0.01526477,0.28827142,0.24460159,-0.26236131
Great Britain,-0.179936091,0.4250059,-0.10884516,-0.21033239,-0.098594107,1.00000000,-0.32891798,-0.33241603,0.04248583,-0.16590704,-0.28508737,0.0630232,-0.20098791,-0.13337226,-0.25089195,0.33371640
Indonesia,-0.137043141,-0.5175476,0.01269456,-0.04680964,0.130174371,-0.32891798,1.00000000,0.51106403,-0.17818432,-0.21861798,0.64013863,-0.2983747,0.24842487,0.08638535,0.35610248,-0.44745331
Japan,-0.132147465,-0.5496629,0.12001132,0.04865095,0.239637490,-0.33241603,0.51106403,1.00000000,-0.46057871,-0.17906811,0.67726220,-0.2998676,0.31269595,0.20131655,0.41898381,-0.47102984
Malaysia,-0.269770549,0.3417002,-0.32225425,-0.36937776,-0.378281741,0.04248583,-0.17818432,-0.46057871,1.00000000,-0.16917057,-0.56450152,0.5649573,-0.10799744,-0.45819149,-0.37749945,0.19810814
Mexico,0.297755939,-0.1303514,0.02640051,0.33912740,-0.085546739,-0.16590704,-0.21861798,-0.17906811,-0.16917057,1.00000000,-0.16602945,-0.1811993,-0.13117037,0.05460400,-0.14463016,0.01854265


In [34]:
cd_sym <- cd_sym %>% pivot_longer(!interests_from, names_to = "origin", values_to = "CS_symm") %>%
                        rename(destination = interests_from)

head(cd_sym)

destination,origin,CS_symm
Argentina,Argentina,1.000000000
Argentina,Australia,-0.162683113
Argentina,Brazil,0.194855413
Argentina,Chile,0.412012736
Argentina,France,-0.003257801
Argentina,Great Britain,-0.179936091


#### nonsymmetric:   
each country is represented by a vector corresponding to the <b>top 50 dishes from origin country</b>

![CDiv](figs/heatmap-similarity-topK-cosine-reds.png)

The **(non-symmetric)** Cultural Similarity between **origin (rows) and destination (columns)** represent how similar origin and destination are in terms of popular dishes from the country of **origin**.   

**Hypothesis:** Immigrants prefer to move to (host) countries culturally similar to their (home/previous) country.

In [35]:
cd_nonsym <- read_csv("data/FB-food-cultural_similarity-nonsymmetric-top-163.csv", col_types = cols())
colnames(cd_nonsym)[1] <- "interests_from"

Warning message:
“Missing column names filled in: 'X1' [1]”

In [36]:
cd_food_o <- cd_nonsym %>% pivot_longer(!interests_from, names_to = "destination", values_to = "CS_nonsymm_food_o") %>%
                           rename(origin = interests_from) %>%
                            merge(cd_sym)

head(cd_food_o)

origin,destination,CS_nonsymm_food_o,CS_symm
Argentina,Argentina,1.00000000,1.000000000
Argentina,Australia,0.30243891,-0.162683113
Argentina,Brazil,0.03255659,0.194855413
Argentina,Chile,0.38357214,0.412012736
Argentina,France,-0.36567789,-0.003257801
Argentina,Great Britain,-0.08462522,-0.179936091


The **(non-symmetric)** Cultural Similarity between **origin (rows) and destination (columns)** representing how similar origin and destination are in terms of popular dishes from the country of **destination**. 

In [37]:
cd_food_d <- cd_nonsym %>% pivot_longer(!interests_from, names_to = "origin", values_to = "CS_nonsymm_food_d") %>%
                            rename(destination = interests_from) %>%
                            merge(cd_sym)

head(cd_food_d)

destination,origin,CS_nonsymm_food_d,CS_symm
Argentina,Argentina,1.00000000,1.000000000
Argentina,Australia,0.30243891,-0.162683113
Argentina,Brazil,0.03255659,0.194855413
Argentina,Chile,0.38357214,0.412012736
Argentina,France,-0.36567789,-0.003257801
Argentina,Great Britain,-0.08462522,-0.179936091


In [38]:
cd <- merge(cd_food_o, cd_food_d)

head(cd)

origin,destination,CS_symm,CS_nonsymm_food_o,CS_nonsymm_food_d
Argentina,Argentina,1.000000000,1.00000000,1.00000000
Argentina,Australia,-0.162683113,0.30243891,-0.36172046
Argentina,Brazil,0.194855413,0.03255659,0.21087156
Argentina,Chile,0.412012736,0.38357214,0.42896829
Argentina,France,-0.003257801,-0.36567789,-0.08601682
Argentina,Great Britain,-0.179936091,-0.08462522,-0.28911818


In [39]:
pca <- read.csv("data/fb-cultural-similarity-pca.csv")

pca_cosine <- as.data.frame(cosine(t(as.matrix(pca[,2:3]))))
colnames(pca_cosine) <- pca$country
pca_cosine["origin"] <- pca$country


pca_cosine_longer <- pivot_longer(pca_cosine, -c("origin"), values_to = "CS_FB_top50_pca", names_to = "destination")
head(pca_cosine_longer)

origin,destination,CS_FB_top50_pca
Argentina,Argentina,1.00000000
Argentina,Australia,0.05466153
Argentina,Brazil,0.79585871
Argentina,Chile,0.98090107
Argentina,France,0.60105840
Argentina,Great Britain,0.37417246


#### Dataset: official statistics + FB cultural distance

In [40]:
official_dataset <- merge(official_dataset, cd)#, mutate(official_dataset, destination = tolower(destination), origin = tolower(origin)))
dataset <- merge(official_dataset, pca_cosine_longer)

head(dataset)

destination,origin,iso_d,iso_o,col,csl,cnl,prox1,lp1,prox2,...,m_flow_mig_rate,m_flow_da_min_open,m_flow_da_min_closed,m_flow_da_pb_closed,CS_wvs,CS_foursquare,CS_symm,CS_nonsymm_food_o,CS_nonsymm_food_d,CS_FB_top50_pca
Argentina,Australia,ARG,AUS,0,0.02058416,0.00080000,0.25,1.945866,0.13052547,...,56,16,105,2224,0.2579213,0.1005334,-0.162683113,-0.36172046,0.30243891,0.05466153
Argentina,Brazil,ARG,BRA,0,0.06052872,0.00080000,0.75,5.837599,0.42439765,...,9997,2332,0,10767,0.7688173,0.8402622,0.194855413,0.21087156,0.03255659,0.79585871
Argentina,Chile,ARG,CHL,1,0.98010004,0.85439998,0.00,0.000000,0.00000000,...,43666,13875,53604,84213,-0.9899189,0.9548769,0.412012736,0.42896829,0.38357214,0.98090107
Argentina,France,ARG,FRA,0,0.13392781,0.01039232,0.50,3.891733,0.25450581,...,1672,467,156,3048,0.1697698,0.8741573,-0.003257801,-0.08601682,-0.36567789,0.60105840
Argentina,Great Britain,ARG,GBR,0,0.08325152,0.00000000,0.25,1.945866,0.13052547,...,193,45,126,1559,0.3229716,0.6263188,-0.179936091,-0.28911818,-0.08462522,0.37417246
Argentina,Indonesia,ARG,IDN,0,0.00000000,0.00000000,0.00,0.000000,0.07436915,...,4,1,4,7,0.4656905,-0.8793284,-0.137043141,-0.21942599,-0.54688918,-0.39869160


In [41]:
dim(dataset)

[1] 240  40

In [41]:
write_csv(dataset, "data/subset-final-dataset-04-22.csv")